# Processing time in the permutation flow shop problem (PFSP)

import libraries

In [3]:
import pandas as pd
import random as rd
import numpy as np
from copy import deepcopy

loading data

In [4]:
df50_20 = pd.read_excel('Dane_PFSP_50_20.xlsx', header=0, index_col=0)
df100_10 = pd.read_excel('Dane_PFSP_100_10.xlsx', header=0, index_col=0)
df200_10 = pd.read_excel('Dane_PFSP_200_10.xlsx', header=0, index_col=0)


Function to calculate time

In [5]:

def Time_sum(data, solution):
    # pad cells with zeros
    tab = np.zeros((len(data), len(data[0])))
    # fil the first row
    for i in np.arange(0, len(data[0])):
        tab[0][i] = data[solution[0]][i] + tab[0][i-1]
    # fill the first column
    for i in np.arange(0, len(data)):
        tab[i][0] = data[solution[i]][0] + tab[i-1][0]
    # fill rest of cells
    for i in np.arange(1, len(data)):
        for j in np.arange(1, len(data[0])):
            tab[i][j] = max(tab[i-1][j], tab[i][j-1]) + data[solution[i]][j]
    # time to go through all jobs on all machines
    return tab[len(tab)-1][len(tab[0])-1]

## Simulated annealing 
(neighborhood type: SWAP)

Parameters: 
T_max - maximal temperature,
T_min - minimal temperature,
alfa - the rate of temperature drop,
iter - number of iterations,
df - data.

In [6]:
def SA (T_max, T_min, alfa, iter, df):

    # convert a data frame into a list (acceleration of calculations)
    lista = df.values.tolist()
    # innitial solution
    list1 = np.arange(0, len(lista)) 
    rd.shuffle(list1)
    list1 = list(list1)
    T = T_max
    
    # stop condition of the algorithm
    while T > T_min: 
        for j in np.arange(0, iter):

            # finding a neighboring solution (type: SWAP)
            rand = False
            while not rand:
                # draw two machines to swap
                r1 = rd.randint(0, len(lista)-1)
                r2 = rd.randint(0, len(lista)-1)
                if r1 != r2:
                    rand = True
            list2 = deepcopy(list1)   # deepcopy
            
            sum, sum2 = 0, 0
            # the sum of the distances for the solutions
            # Obliczenie sum czasów dla obu rozwiązań
            suma, suma2 = 0, 0
            suma = Time_sum(lista, list1)
            suma2 = Time_sum(lista, list2)
            # checking whether the new solution is better
            if sum > sum2:
                list1 = deepcopy(list2)
            else:      # condition for accepting a worse solution
                if np.exp((sum - sum2)/T) > rd.random():
                    list1 = deepcopy(list2)
        T = T*alfa
    sum_min = 0
    
    # Printing the parameter values, the order of the tasks and the distance obtained
    print("Parametres: ", '\n',"Number od tasks:",len(lista), '\n', 'Number of machines: ', len(lista[0]), '\n', 'Type of neighbourhood: SWAP', '\n', 'Temperature max: ', T_max, '\n', 'Temperature min: ', T_min, '\n', 'Alfa: ', alfa, '\n', 'Number of iterations: ', iter, '\n')
    sum_min = sum_min + Time_sum(lista, list1)
    print('RESULTS', '\n', list1, '\n', 'Total time: ', sum_min, '\n')


Test

In [7]:
SA(100,1,0.5,100,df100_10)

Parametres:  
 Number od tasks: 100 
 Number of machines:  10 
 Type of neighbourhood: SWAP 
 Temperature max:  100 
 Temperature min:  1 
 Alfa:  0.5 
 Number of iterations:  100 

RESULTS 
 [96, 28, 68, 66, 58, 13, 7, 54, 61, 87, 95, 57, 1, 29, 36, 32, 76, 71, 40, 43, 26, 55, 45, 75, 25, 50, 86, 89, 65, 91, 22, 16, 59, 83, 53, 2, 48, 63, 3, 37, 69, 85, 41, 88, 80, 78, 18, 90, 74, 64, 20, 14, 6, 5, 94, 77, 21, 19, 15, 34, 46, 97, 51, 11, 12, 73, 10, 49, 92, 33, 99, 81, 4, 9, 8, 39, 47, 38, 98, 31, 17, 72, 93, 42, 82, 44, 56, 62, 27, 30, 67, 24, 60, 52, 84, 23, 35, 70, 79, 0] 
 Total time:  6796.0 



## Simulated annealing 2
(neighborhood type: inversion)

Parameters: 
T_max - maximal temperature,
T_min - minimal temperature,
alfa - the rate of temperature drop,
iter - number of iterations,
df - data.

In [8]:
def SA_inversion (T_max, T_min, alfa, iter, df):

    # Zamiana ramki danych na listę (przyspieszenie obliczeń)
    lista = df.values.tolist()
    # innitial solution
    list1 = np.arange(0, len(lista)) 
    rd.shuffle(list1)
    list1 = list(list1)
    T = T_max
    
    # stop condition of the algorithm
    while T > T_min: 
        for j in np.arange(0, iter):

            # finding a neighboring solution (type: SWAP)
            rand = False
            while not rand:
                # draw two machines to swap
                r1 = rd.randint(0, len(lista)-1)
                r2 = rd.randint(0, len(lista)-1)
                if r1 != r2:
                    rand = True
            list2 = deepcopy(list1)   # deepcopy
            list2[r1] = list1[r2]
            list2[r2] = list1[r1]   
            if r1 < r2:
                list2[r1:r2] = reversed(list2[r1:r2])
            else:
                list2[r2:r1] = reversed(list2[r2:r1])  # list2 - new, neighboring solution
            sum, sum2 = 0, 0
            # the sum of the distances for the solutions
            # Obliczenie sum czasów dla obu rozwiązań
            suma, suma2 = 0, 0
            suma = Time_sum(lista, list1)
            suma2 = Time_sum(lista, list2)
            # checking whether the new solution is better
            if sum > sum2:
                list1 = deepcopy(list2)
            else:      # condition for accepting a worse solution
                if np.exp((sum - sum2)/T) > rd.random():
                    list1 = deepcopy(list2)
        T = T*alfa
    sum_min = 0
    
    # Printing the parameter values, the order of the tasks and the distance obtained
    print("Parametres: ", '\n',"Number od tasks: ",len(lista), '\n', 'Number of machines: ', len(lista[0]), '\n', 'Type of neighbourhood: SWAP', '\n', 'Temperature max: ', T_max, '\n', 'Temperature min: ', T_min, '\n', 'Alfa: ', alfa, '\n', 'Number of iterations: ', iter, '\n')
    
    sum_min = sum_min + Time_sum(lista, list1)
    print('RESULTS', '\n', list1, '\n', 'Total time: ', sum_min, '\n')

Test

In [9]:
SA_inversion(100,1,0.5,100, df100_10)

Parametres:  
 Number od tasks:  100 
 Number of machines:  10 
 Type of neighbourhood: SWAP 
 Temperature max:  100 
 Temperature min:  1 
 Alfa:  0.5 
 Number of iterations:  100 

RESULTS 
 [76, 26, 35, 4, 27, 59, 1, 90, 45, 10, 16, 69, 71, 99, 7, 43, 24, 46, 82, 15, 42, 83, 13, 22, 9, 0, 78, 30, 5, 18, 62, 91, 23, 36, 28, 65, 6, 86, 44, 33, 25, 73, 94, 63, 55, 47, 85, 52, 2, 20, 54, 29, 56, 19, 21, 67, 31, 38, 32, 40, 97, 41, 51, 84, 98, 88, 74, 11, 79, 53, 72, 3, 87, 49, 81, 8, 70, 89, 61, 96, 50, 12, 60, 37, 48, 17, 93, 80, 95, 75, 77, 57, 92, 64, 58, 66, 39, 34, 14, 68] 
 Total time:  6623.0 



## Hill Climbing

In [10]:
# function to use chosen neighborhood
def find_neigbor(list1, list2, r1, r2, neigborhood):
    
    if neigborhood == "SWAP":
        list2[r1] = list1[r2]
        list2[r2] = list1[r1]   # list2 - new, neighboring solution
    if neigborhood == "inversion":
        if r1 < r2:
            list2[r1:r2] = reversed(list2[r1:r2])
        else:
            list2[r2:r1] = reversed(list2[r2:r1])  # list2 - new, neighboring solution
    
    return list2

In [12]:
def HillClimbing (iter, iter_no_improve, lista, neigborhood):

    
    # convert a data frame into a list (acceleration of calculations)
    lista = lista.values.tolist()
    # list1 - innitial solution
    list1 = np.arange(0, len(lista)) 
    rd.shuffle(list1)
    list1 = list(list1)

    k = 0
    while k == iter_no_improve or k == iter:

        # find a neighboring solution
        rand = False
        while not rand:
            # draw two cities to swap
            r1 = rd.randint(0, len(lista)-1)
            r2 = rd.randint(0, len(lista)-1)
            if r1 != r2:
                rand = True
        list2 = deepcopy(list1)   # deepcopy

            # function to use chosen neighborhood
        find_neigbor(list1, list2, r1, r2, neigborhood, lista)
            
        sum, sum2 = 0, 0
        # the sum of the distances for the solutions
        for i in np.arange(0, len(lista)):
            sum = sum + lista.iloc[list1[i - 1], list1[i]]
            sum2 = sum2 + lista.iloc[list2[i - 1], list2[i]]
        # checking whether the new solution is better
        if sum > sum2:
            list1 = deepcopy(list2)
        if sum < sum2:
            k = k+1
          
        
    sum_min = 0
    # Printing the parameter values, the order of the cities and the distance obtained
    print("Parametres: ", '\n', "Number od tasks:", len(lista), '\n', 'Number of machines: ', len(lista[0]), '\n', 'Type of neighbourhood:',neigborhood ,'\n', 'Iteration without improvement: ', iter_no_improve,'\n', 'Number of iterations: ', iter, '\n')
    sum_min = sum_min + Time_sum(lista, list1)
    print('RESULTS', '\n', list1, '\n', 'Total time: ', sum_min, '\n')  
         

Test

In [13]:
print("====== test 1 ============")
HillClimbing(10,10,df100_10, "inversion")

====== test 1 ============
Parametres:  
 Number od tasks: 100 
 Number of machines:  10 
 Type of neighbourhood: inversion 
 Iteration without improvement:  10 
 Number of iterations:  10 

RESULTS 
 [64, 14, 97, 30, 28, 0, 88, 12, 54, 22, 38, 41, 11, 48, 74, 69, 7, 81, 33, 67, 52, 3, 8, 76, 2, 24, 27, 94, 49, 79, 73, 71, 45, 60, 93, 89, 44, 96, 90, 63, 92, 16, 17, 78, 82, 29, 83, 51, 72, 36, 21, 68, 62, 61, 58, 37, 20, 57, 75, 43, 39, 95, 77, 55, 80, 53, 59, 50, 98, 19, 66, 42, 85, 70, 40, 9, 65, 26, 1, 6, 23, 10, 4, 13, 91, 47, 87, 84, 15, 32, 31, 46, 35, 34, 86, 18, 5, 56, 25, 99] 
 Total time:  6507.0 

